<a href="https://colab.research.google.com/github/sophielouie/beer-recommendation-system/blob/main/GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy import spatial
import statistics as stats
import math
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split
from networkx.algorithms.dag import topological_sort
import networkx as nx
import plotly.graph_objects as go
from node2vec import Node2Vec

In [8]:
#import Node2Vec and Hybrid Notebooks
import node2vec_module

we want to import functions into our gui file
- transport endpoint is not connected
- we do not understand google file explorer

In [ ]:
!pip install node2vec

In [ ]:
!cp /content/drive/MyDrive/Recommender\ System/node2vec.py /content

In [10]:
import node2vec as n

In [15]:
df = n.clean_beer_reviews()
# not working that is why clean_beer_reviews copied in

AttributeError: module 'node2vec' has no attribute 'clean_beer_reviews'

# app interface

In [21]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Dropdown
from ipywidgets import interact, interactive, fixed, interact_manual
import hybridmodel
from hybridmodel import clean_beer_reviews
import pandas as pd

In [ ]:
# paragraph describing each model

In [16]:
def clean_beer_reviews():
    # storing beer review dataset
    beer_reviews = pd.read_csv("beer_reviews.csv")
    # creating a unique identifier for each beer using brewery name and beer name
    beer_reviews['Unique Beer Name'] = beer_reviews['brewery_name'] + ' ' + beer_reviews['beer_name']
    # storing beer profile dataset
    beer_profile = pd.read_csv("beer_profile_and_ratings.csv")
    # columns to drop from beer reviews
    drop_cols = ['brewery_id', 'brewery_name',  'beer_name', 'beer_abv', 'beer_beerid']
    # dropping columns from beer reviews
    beer_reviews.drop(columns = drop_cols, inplace = True)
    # columns to drop from  beer profile
    drop_cols = ['Name', 'Style', 'Brewery', 'Description',
                 'Min IBU', 'Max IBU', 'Alcohol', 'review_aroma', 'review_appearance', 'review_palate',
                 'review_taste', 'review_overall', 'number_of_reviews']
    # dropping columns from beer profile
    beer_profile.drop(columns = drop_cols, inplace = True)
    # combining beer review and beer profile datasets to have profile of each beer attached to every review
    df_beer = pd.merge(beer_reviews, beer_profile, left_on = 'Unique Beer Name', right_on = 'Beer Name (Full)', how = 'inner')
    # isolating the numerical columns that need to be scaled
    need_scaling = df_beer.drop(columns = ['review_time', 'review_profilename', 'beer_style', 'Unique Beer Name', 'Beer Name (Full)'])
    # storing the informational portion of the dataset that does not need scaling
    informational = df_beer[['review_time', 'review_profilename', 'beer_style', 'Unique Beer Name', 'Beer Name (Full)']]
    # renaming beer name column
    informational.rename(columns = {'Beer Name (Full)': 'Beer Name'}, inplace = True)

    # scaling the data
    scaler = MinMaxScaler()
    scaler.fit(need_scaling)
    need_scaling = pd.DataFrame(scaler.transform(need_scaling), columns = need_scaling.columns)

    # recombining the informational data and scaled data
    df = pd.concat([informational, need_scaling], axis = 1)
    return df

In [17]:
BEER_DF = clean_beer_reviews()

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [28]:
# make a fucntion that returns a list of beers
def make_beer_list():
    return BEER_DF["Beer Name"].unique()


# list of user, beer combination
def make_beer_user_list(user):
    return BEER_DF.loc[BEER_DF["review_profilename"] == user]["Beer Name"]

# list of users to choose from
def make_user_list():
    return BEER_DF['review_profilename'].unique()

In [29]:
print(len(make_beer_list()))

3183


In [43]:
# MDOEL TYPE DROPDOWN TO USER/BEER OPTIONS
def model_dropdown_eventhandler(change):
    determine(model_dropdown.value)

option_list = ["Hybrid", "Node2Vec"]

user = make_user_list()[:100]
beer = make_beer_list()

def determine(x):
    beer_dropdown.layout.visibility = "hidden"
    dropdown_dep.index = None
    if x == "Hybrid":
        dropdown_dep.options = user
    else:
        dropdown_dep.options = beer
        
# HYBRID MODEL USER CHOISE TO CORRESPONDING BEER OPTIONS
def beer_dropdown_eventhandler(change):
    determine_beer(dropdown_dep.value)

def determine_beer(x):
    beer_dropdown.index = None
    beer_user = make_beer_user_list(x)
    if len(beer_user) > 0:
        beer_dropdown.options = beer_user
        beer_dropdown.layout.visibility = "visible"
    else: 
        beer_dropdown.layout.visibility = "hidden"

model_dropdown = Dropdown(description="Select a model", options=option_list)

dropdown_dep = Dropdown(description="Choose one")
model_dropdown.observe(model_dropdown_eventhandler, names='value')

beer_dropdown = Dropdown(description="Choose one", layout = {"visibility" : "hidden"})
dropdown_dep.observe(beer_dropdown_eventhandler, names='value')


display(model_dropdown, dropdown_dep, beer_dropdown)

Dropdown(description='Select a model', options=('Hybrid', 'Node2Vec'), value='Hybrid')

Dropdown(description='Choose one', options=(), value=None)

Dropdown(description='Choose one', layout=Layout(visibility='hidden'), options=(), value=None)

In [48]:
widgets.IntSlider(
    value=5,
    min=0,
    max=10,
    step=1,
    description='Number of recommendations:',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='d'
)

IntSlider(value=5, continuous_update=False, description='Number of recommendations:', max=10, orientation='ver…

In [45]:
button = widgets.Button(description="Run model")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        

button.on_click(on_button_clicked)

Button(description='Run model', style=ButtonStyle())

Output()

In [19]:
model_type = widgets.Dropdown(options=["Hybrid", "Node2Vec"], description="Select a Model")
display(model_type)

Dropdown(description='Select a Model', options=('Hybrid', 'Node2Vec'), value='Hybrid')

In [ ]:
def choose_model(Model):
    list_of_user = make_user_list()
    chosen_user = widgets.Dropdown(options=list_of_user[:10], description="Select a User",
                                  layout = {"visibility" : "hidden"})
    if Model == "Hybrid":
        # then display user and beer options
        chosen_user.layout.visibility = "visible"
        display(chosen_user)
        user = chosen_user.value 
    else:
        # only display beer
        list_of_beer = make_beer_list()
        n2v_beer = widgets.Dropdown(options=list_of_beer[:10], description="Select a Beer")
        display(n2v_beer)


        

w = interactive(choose_model, Model = model_type.value)
display(w)

In [ ]:
def choose_user_beer(User):
    if (model_type.value == 'Hybrid'):
        #create user dropdown
    list_of_beer = make_beer_user_list(user)
    hybrid_beer = widgets.Dropdown(options=list_of_beer[:10], description="Select a Beer")
    display(hybrid_beer)
p = interactive(choose_user_beer, User)

In [ ]:
btn = widgets.Button(description='Start')
display(btn)
def btn_eventhandler(obj):
    # call model function
    print("return model result")
btn.on_click(btn_eventhandler)